In [ ]:
pip install transformers datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("samaxr/code-summary-java")
print(dataset)
train_data = dataset["train"]
subset = train_data.select(range(50))  # Select first 200 indices

# for item in subset:
#     code = item["code"]
#     summary = item["summary"]
    # ... (process code and summary)


DatasetDict({
    train: Dataset({
        features: ['code', 'summary'],
        num_rows: 285670
    })
    validation: Dataset({
        features: ['code', 'summary'],
        num_rows: 31741
    })
    test: Dataset({
        features: ['code', 'summary'],
        num_rows: 79352
    })
})


In [ ]:
len(train_data['code'])

285670

In [ ]:
# from transformers import BertTokenizer

# # Define tokenizer for sentence processing (can be replaced with other tokenizers)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



# def preprocess_data(data):
#     """
#     Preprocesses code-summary pairs by tokenizing them and converting to numerical representation.

#     Args:
#        data: A list of dictionaries containing 'code' and 'summary' keys.

#     Returns:
#        A list of dictionaries with 'code_ids', 'code_mask', 'summary_ids', and 'summary_mask' keys.
#     """
#     processed_data = []  # Limit to first 200 elements
#     for item in data:
#         code = item['code']
#         summary = item['summary']

#         # Tokenize code and summary
#         code_tokens = tokenizer.encode(code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
#         summary_tokens = tokenizer.encode(summary, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

#         # Add processed data to list
#         processed_data.append({
#             'code_ids': code_tokens,
#             'code_mask': [1] * len(code_tokens),
#             'summary_ids': summary_tokens,
#             'summary_mask': [1] * len(summary_tokens)
#         })
#     return processed_data

# processed_data = preprocess_data(dataset["train"])  # Directly use training split

# print(processed_data)
# Now you can use 'processed_data' to train your code summarization model


In [ ]:
from transformers import BertTokenizer
import concurrent.futures

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define function to preprocess a batch of data
def preprocess_batch(item):
    processed_batch = []
    code = item['code']
    summary = item['summary']

    # Tokenize code and summary
    code_tokens = tokenizer.encode(code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
    summary_tokens = tokenizer.encode(summary, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

    # Add processed data to list
    processed_batch.append({
        'code_ids': code_tokens,
        'code_mask': [1] * len(code_tokens),
        'summary_ids': summary_tokens,
        'summary_mask': [1] * len(summary_tokens)
    })

    '''
    for item in batch:
        code = item['code']
        summary = item['summary']

        # Tokenize code and summary
        code_tokens = tokenizer.encode(code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
        summary_tokens = tokenizer.encode(summary, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

        # Add processed data to list
        processed_batch.append({
            'code_ids': code_tokens,
            'code_mask': [1] * len(code_tokens),
            'summary_ids': summary_tokens,
            'summary_mask': [1] * len(summary_tokens)
        })
    '''
    return processed_batch


# Define function to preprocess data in batches using multiprocessing
def preprocess_data_multiprocessed(data, batch_size):
    processed_data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # data_list = list(data)
        #train_data = data["train"]

        # Divide the data into batches
        batches = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]
        # print(type(batches))
        # for item in batches:
        #   print(item)
        #   print(len(item['code']))

        # Preprocess each batch using multiple processes
        for processed_batch in executor.map(preprocess_batch, batches):
            processed_data.extend(processed_batch)
    return processed_data


# Set batch size
batch_size = 16

# Preprocess data in batches
# print(type(subset))
# print(subset.select(range(50)))
processed_data = preprocess_data_multiprocessed(train_data, batch_size)

print("Processed")

#print(processed_data)
# Now you can use 'processed_data' to train your code summarization model

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Processed


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm.auto import tqdm


In [ ]:
# Define the model and tokenizer
model_name = "t5-small"  # You can use any pre-trained T5 model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define dataset class
class CodeSummaryDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Convert processed_data into input format required by T5 model
def prepare_input_data(processed_data):
    input_data = []
    for entry in processed_data:
        code = entry['code']
        summary = entry['summary']
        input_text = f"code: {code} summary: {summary}"  # Concatenate code and summary
        input_data.append(input_text)
    return input_data

# Prepare training data
processed_data = prepare_input_data(train_data)
train_dataset = CodeSummaryDataset(processed_data)

# Define training parameters
batch_size = 16  # Increased batch size (you can try 8 as well)
epochs = 10  # Increased epochs
accumulation_steps = 4  # Accumulate gradients over 4 steps

# Define training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    tqdm_dataloader = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")

    for i, batch in enumerate(tqdm_dataloader):
        input_ids = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512, add_special_tokens=True).input_ids.to(device)
        labels = input_ids.clone()
        labels[input_ids == tokenizer.pad_token_id] = -100  # Ignore padding tokens in loss calculation
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # Perform gradient accumulation
        loss = loss / accumulation_steps
        loss.backward()

        # Update parameters after accumulating gradients for specified steps
        if (i + 1) % accumulation_steps == 0 or i == len(train_dataloader) - 1:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item()

    tqdm_dataloader.close()
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")

# Save the trained model
model.save_pretrained("code_summary_model")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Epoch 1/10:   0%|          | 0/17855 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1004.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 483.06 MiB is free. Process 7592 has 14.27 GiB memory in use. Of the allocated memory 13.69 GiB is allocated by PyTorch, and 466.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [ ]:
model_name = "Salesforce/codet5-small"  # Switched to a smaller model variant
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Convert processed_data into input format required by CodeT5 model
def prepare_input_data(processed_data):
    input_data = []
    for entry in processed_data:
        code = entry['code']
        summary = entry['summary']
        input_text = f"code: {code} \n summary: {summary}"
        input_data.append(input_text)
    return input_data

class CodeSummaryDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Prepare training data
train_data=train_data.select(range(10000))
processed_data = prepare_input_data(train_data)
train_dataset = CodeSummaryDataset(processed_data)


# Define training parameters
batch_size = 8
epochs = 5
accumulation_steps = 16
max_grad_norm = 1.0

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)

scaler = torch.cuda.amp.GradScaler()

for epoch in range(epochs):
    model.train()
    total_loss = 0
    tqdm_dataloader = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")

    for i, batch in enumerate(tqdm_dataloader):
        input_texts = batch
        input_ids = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=128, add_special_tokens=True).input_ids.to(device)  # Reduced max_length to 128
        labels = input_ids.clone()
        labels[input_ids == tokenizer.pad_token_id] = -100

        with torch.cuda.amp.autocast():
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss

        loss = loss / accumulation_steps
        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0 or i == len(train_dataloader) - 1:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()

    tqdm_dataloader.close()
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")

# Save the trained model
model.save_pretrained("code_summary_model")
# Save the tokenizer
tokenizer.save_pretrained("code_summary_model")


Epoch 1/5: 100%|██████████| 1250/1250 [01:42<00:00, 12.24it/s]


Epoch 1/5, Average Loss: 0.0327


Epoch 2/5: 100%|██████████| 1250/1250 [01:41<00:00, 12.34it/s]


Epoch 2/5, Average Loss: 0.0012


Epoch 3/5: 100%|██████████| 1250/1250 [01:40<00:00, 12.41it/s]


Epoch 3/5, Average Loss: 0.0006


Epoch 4/5: 100%|██████████| 1250/1250 [01:41<00:00, 12.30it/s]


Epoch 4/5, Average Loss: 0.0004


Epoch 5/5: 100%|██████████| 1250/1250 [01:41<00:00, 12.30it/s]


Epoch 5/5, Average Loss: 0.0003


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(processed_data, test_size=5, random_state=42)

# Prepare training data
train_input_data = prepare_input_data(train_data)
train_dataset = CodeSummaryDataset(train_input_data)

# Prepare testing data
test_input_data = prepare_input_data(test_data)
test_dataset = CodeSummaryDataset(test_input_data)

# Define training and testing dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
for epoch in range(epochs):
    # Training phase
    model.train()
    train_loss = 0
    tqdm_dataloader = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")

    for batch in tqdm_dataloader:
        # Process batch
        input_ids = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512, add_special_tokens=True).input_ids.to(device)
        labels = input_ids.clone()
        labels[input_ids == tokenizer.pad_token_id] = -100

        # Forward pass
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update loss
        train_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = train_loss / len(train_dataloader)
    tqdm_dataloader.close()

    # Testing phase
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in test_dataloader:
            # Process batch
            input_ids = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512, add_special_tokens=True).input_ids.to(device)
            labels = input_ids.clone()
            labels[input_ids == tokenizer.pad_token_id] = -100

            # Forward pass
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss

            # Update loss
            test_loss += loss.item()

    # Calculate average testing loss
    avg_test_loss = test_loss / len(test_dataloader)

    # Print training and testing loss for the epoch
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

# Save the trained model
model.save_pretrained("code_summary_model")


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the trained model
model_path = "code_summary_model"  # Path to the saved model directory
try:
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
except Exception as e:
    print("Error loading model:", e)
    raise

# Function to generate summaries
def generate_summary(code_snippet):
    try:
        input_text = f"code: {code_snippet} summary:"
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True, padding='max_length')
        output_ids = model.generate(input_ids.to(model.device))
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return summary
    except Exception as e:
        print("Error generating summary:", e)
        return None

# Example usage
sample_code = """
def bubble_sort(arr):
    n = len(arr)
    for i in range(n-1):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr
"""

summary = generate_summary(sample_code)
if summary:
    print("Generated Summary:", summary)
else:
    print("Failed to generate summary.")

In [ ]:
pip install rouge nltk

In [ ]:
pip install Spice

In [ ]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

# ... load model, generate summaries for samples ...

# Calculate metrics
rouge = Rouge()
bleu_scores = []
meteor_scores = []

for code, generated_summary, reference_summary in zip(samples, generated_summaries, reference_summaries):
    rouge_scores = rouge.get_scores(generated_summary, reference_summary)
    bleu_scores.append(sentence_bleu([reference_summary], generated_summary))
    meteor_scores.append(meteor_score([reference_summary], generated_summary))

# Print final scores
print("ROUGE Scores:", rouge_scores)
print("BLEU Scores:", bleu_scores)
print("METEOR Scores:", meteor_scores)
